In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LinearRegression

In [2]:
%load_ext autoreload
%autoreload 2
from functions import loaddata, MACD, RSI
from portfolio import Portfolio

In [3]:
def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return 0
    else:
        return 2
    
def my_log(x):
    if x != 0 and np.isnan(x) == False:
        return np.log(x)
    else:
        return x

In [34]:
start_date = 20170101
end_date = 20190101
rtxm_ti2 = loaddata("rtxm_ti2", start_date, end_date)
rtxm_ti1 = loaddata("rtxm_ti1", start_date, end_date)
r_ti2 = loaddata("r_ti2", start_date, end_date)
volume = loaddata("volall_day", start_date, end_date)
mid_close = loaddata("mid_close", start_date, end_date)
mid_open = loaddata("mid_open", start_date, end_date)

start_date1 = 20190101
end_date1 = 20200101
rtxm_ti2_test = loaddata("rtxm_ti2", start_date1, end_date1)
rtxm_ti1_test = loaddata("rtxm_ti1", start_date1, end_date1)
r_ti2_test = loaddata("r_ti2", start_date1, end_date1)
volume_test = loaddata("volall_day", start_date1, end_date1)
mid_close_test = loaddata("mid_close", start_date1, end_date1)
mid_open_test = loaddata("mid_open", start_date1, end_date1)

In [35]:
total = []
total1 = []
chk = []
test = []
pls = []

for i in range(mid_close.shape[0]):
    total_made = 0
    #TRAINING DATA
    price_i = mid_close.iloc[i]
    overnight_i = rtxm_ti1.iloc[i, 1:]
    intraday_i = rtxm_ti2.iloc[i,:-1]
    volume_i = volume.iloc[i, :-1].map(lambda x: my_log(x), na_action="ignore")
    rsi_12 = RSI(rtxm_ti2.iloc[i], 12)[:-1]
    macd_26_12_9 = MACD(price_i, 26, 12, 9)[:-1]
    y = r_ti2.iloc[i, 1:].map(lambda x: sign(x), na_action="ignore")
    data = {'X1': np.array(overnight_i), 'X2': np.array(intraday_i), "X3": np.array(volume_i), 
            "X4": np.array(rsi_12), "X5": np.array(macd_26_12_9),
            "Y": np.array(y)}
    data_train = pd.DataFrame(data)
    new_train = data_train.dropna()
    temp = new_train.sample(frac=1).reset_index(drop=True)
    X_train = temp[["X1","X2","X3","X4","X5"]]
    y_train = temp[["Y"]]

    #TESTING DATA
    price_i_test = mid_close_test.iloc[i]
    open_price_i_test = mid_open_test.iloc[i]
    overnight_i_test = rtxm_ti1_test.iloc[i, 1:]
    intraday_i_test = rtxm_ti2_test.iloc[i,:-1]
    volume_i_test = volume_test.iloc[i, :-1].map(lambda x: my_log(x), na_action="ignore")
    rsi_12_test = RSI(rtxm_ti2_test.iloc[i], 12)[:-1]
    macd_26_12_9_test = MACD(price_i_test, 26, 12, 9)[:-1]
    y_test = r_ti2_test.iloc[i, 1:].map(lambda x: sign(x), na_action="ignore")
    data_test = {'X1': np.array(overnight_i_test), 'X2': np.array(intraday_i_test), "X3": np.array(volume_i_test), 
        "X4": np.array(rsi_12_test), "X5": np.array(macd_26_12_9_test),
        "Y": np.array(y_test)}
    data_test = pd.DataFrame(data_test, index=y_test.index)
    new_test = data_test.dropna()
    X_test = new_test[["X1","X2","X3","X4","X5"]]  
    y_test = new_test[["Y"]]

    #Train and Test individual models
    if new_train.shape[0] > 100 and new_test.shape[0] > 100:
        LR = LogisticRegression(max_iter=1000)

        LR.fit(X_train, np.ravel(y_train))
        total1.append((i, LR))
        chk.append(LR.score(X_test, y_test))

        signals = LR.predict(X_test)
        count = 0

        signal_dates = new_test.index

        for j in range(len(signals)):
            price1 = open_price_i_test.loc[signal_dates[j]]
            price2 = price_i_test.loc[signal_dates[j]]
            pls.append(int(price1 - price2 < 0) == np.array(y_test)[j][0])
            #Short
            if signals[j] == 0:
                gain = price1 - price2
                test.append(gain > 0)
                total_made += gain
            #Long
            elif signals[j] == 1:
                gain = price2 - price1
                test.append(gain > 0)
                total_made += gain
        total.append(total_made)


print("Proportion of correct labels: ", sum(pls)/len(pls))
print("Proportion of correct guesses: ", sum(test)/len(test))
print("Total Money Made: ", sum(total))
print("\n")
print("Mean Score: ", sum(chk)/len(chk))
print("Max Score: ", max(chk))
print("Min Score: ", min(chk))

/Users/jameshetherington/anaconda3/envs/ml-0451/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Proportion of correct labels:  0.9467098037343011
Proportion of correct guesses:  0.49094517430211093
Total Money Made:  4686.785820081868


Mean Score:  0.506868323945863
Max Score:  0.6556016597510373
Min Score:  0.4074074074074074


In [17]:
print(sum(chk)/len(chk), max(chk), min(chk))

0.5083207288154992 0.6514522821576764 0.4074074074074074


In [18]:
print("Mean Score: ", sum(chk)/len(chk))
print("Max Score: ", max(chk))
print("Min Score: ", min(chk))

Mean Score:  0.5083207288154992
Max Score:  0.6514522821576764
Min Score:  0.4074074074074074


In [36]:
sum(total) / mid_open_test.iloc[:,0].dropna().sum()

0.1151190161020845